<h3> Training a Neural Network for calculating portfolio weights <h3>

In [13]:
#Simulate data

import warnings
import pandas as pd
import numpy as np

# Suppress warnings
warnings.filterwarnings("ignore")


# Simulate historical stock prices for a week for demonstration
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
initial_prices = [150, 250, 2800, 3400, 700]
start_date = '2023-10-01'
end_date = '2023-10-31'

# Function to simulate stock prices for a week
def simulate_stock_prices(symbols, start_date, end_date, initial_prices, mu=0.0005, sigma=0.01):
    dates = pd.date_range(start=start_date, end=end_date, freq='B')
    stock_prices = pd.DataFrame(index=dates, columns=symbols)

    for i, symbol in enumerate(symbols):
        stock_prices[symbol].iloc[0] = initial_prices[i]
        for t in range(1, len(dates)):
            random_return = np.random.normal(loc=mu, scale=sigma)
            stock_prices[symbol].iloc[t] = stock_prices[symbol].iloc[t - 1] * np.exp(random_return)

    return stock_prices

# Simulate stock prices
portfolio_full = simulate_stock_prices(symbols, start_date, end_date, initial_prices)

# Calculate daily returns
portfolio_returns_full = portfolio_full.pct_change().dropna()

In [14]:
import warnings
import pandas as pd
import numpy as np

# Suppress warnings
warnings.filterwarnings("ignore")

# Set look-back period
look_back = 2

# Initialize lists for final weights and returns
weights_final = []
ret = []

# Loop through the portfolio data
for i in np.arange(look_back, portfolio_full['AAPL'].shape[0], 1):
    # Select the portfolio for the look-back period
    portfolio = portfolio_full.iloc[i - look_back:i]
    
    # Calculate the covariance matrix for the returns of the portfolio
    var_matrix = portfolio_returns_full.iloc[i - look_back:i].cov()
    
    # Append the previous day's returns
    ret.append(portfolio_returns_full.iloc[i - 1:i].values.flatten())

    port_returns = []
    port_volatility = []
    port_weights = []

    num_assets = len(portfolio.columns)
    num_portfolios = 5000

    # Calculate daily mean returns over the look-back period
    individual_rets = portfolio_returns_full.iloc[i - look_back:i].mean()

    # Simulate portfolios
    for port in range(num_portfolios):
        # Generate random weights and normalize them
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)

        # Calculate portfolio return and volatility
        returns = np.dot(weights, individual_rets)
        port_returns.append(returns)

        # Compute the portfolio variance
        var = var_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()
        
        # Calculate daily standard deviation (volatility)
        sd = np.sqrt(var)
        port_volatility.append(sd)
        
        # Store weights
        port_weights.append(weights)

    # Create a DataFrame to store the results
    data = {'Returns': port_returns, 'Volatility': port_volatility}

    # Store weights for each asset
    for counter, symbol in enumerate(portfolio.columns.tolist()):
        data[symbol + ' weight'] = [w[counter] for w in port_weights]

    portfolios_V1 = pd.DataFrame(data)

    # Find the portfolio with the minimum volatility
    min_vol_port = portfolios_V1.iloc[portfolios_V1['Volatility'].idxmin()]
    weights = min_vol_port[2:]  # Exclude Returns and Volatility

    # Append the final weights of the minimum volatility portfolio
    weights_final.append(weights.values)

    # Print progress
    print("Iteration " + str(i - look_back + 1) + " completed")

Iteration 1 completed
Iteration 2 completed
Iteration 3 completed
Iteration 4 completed
Iteration 5 completed
Iteration 6 completed
Iteration 7 completed
Iteration 8 completed
Iteration 9 completed
Iteration 10 completed
Iteration 11 completed
Iteration 12 completed
Iteration 13 completed
Iteration 14 completed
Iteration 15 completed
Iteration 16 completed
Iteration 17 completed
Iteration 18 completed
Iteration 19 completed
Iteration 20 completed


In [16]:
# Convert weights_final to a NumPy array and reshape it
weights_final = np.array(weights_final)  # Convert to NumPy array
# Reshape to (num_iterations, num_assets)
weights_final = weights_final.reshape(weights_final.shape[0], -1)  # -1 automatically calculates the size

# Convert ret to a NumPy array and reshape it
ret = np.array(ret)  # Convert to NumPy array
# Reshape to (num_iterations, num_assets)
ret = ret.reshape(ret.shape[0], -1)  # -1 automatically calculates the size

In [19]:
#Neural network training for real-time portfolio optimization

import keras
from keras.layers import Dense, Activation
from keras.models import Sequential

model = Sequential()
model.add(Dense(128, input_dim=5, activation='relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

2024-10-15 23:40:56.271397: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
model.fit(ret, weights_final,batch_size=32, epochs=500,verbose=1)

Epoch 1/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0441 - mean_absolute_error: 0.0441
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0439 - mean_absolute_error: 0.0439
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0439 - mean_absolute_error: 0.0439
Epoch 4/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0440 - mean_absolute_error: 0.0440
Epoch 5/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0436 - mean_absolute_error: 0.0436
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0436 - mean_absolute_error: 0.0436
Epoch 7/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0435 - mean_absolute_error: 0.0435
Epoch 8/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0437 - mean_absolute_error: 0.0437
Epoch 9/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0435 - mean_absolute_error: 0.0435
Epoch 10/500
1/1 [=====

1/1 [==============================] - 0s 7ms/step - loss: 0.0416 - mean_absolute_error: 0.0416
Epoch 77/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0421 - mean_absolute_error: 0.0421
Epoch 78/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0413 - mean_absolute_error: 0.0413
Epoch 79/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0431 - mean_absolute_error: 0.0431
Epoch 80/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0423 - mean_absolute_error: 0.0423
Epoch 81/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0432 - mean_absolute_error: 0.0432
Epoch 82/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0427 - mean_absolute_error: 0.0427
Epoch 83/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0428 - mean_absolute_error: 0.0428
Epoch 84/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0432 - mean_absolute_error: 0.0432
Epoch 85/500
1/1 [=============

1/1 [==============================] - 0s 5ms/step - loss: 0.0406 - mean_absolute_error: 0.0406
Epoch 151/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0415 - mean_absolute_error: 0.0415
Epoch 152/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0407 - mean_absolute_error: 0.0407
Epoch 153/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0412 - mean_absolute_error: 0.0412
Epoch 154/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0408 - mean_absolute_error: 0.0408
Epoch 155/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0408 - mean_absolute_error: 0.0408
Epoch 156/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0404 - mean_absolute_error: 0.0404
Epoch 157/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0406 - mean_absolute_error: 0.0406
Epoch 158/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0410 - mean_absolute_error: 0.0410
Epoch 159/500
1/1 [=====

1/1 [==============================] - 0s 8ms/step - loss: 0.0385 - mean_absolute_error: 0.0385
Epoch 225/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0383 - mean_absolute_error: 0.0383
Epoch 226/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0385 - mean_absolute_error: 0.0385
Epoch 227/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0382 - mean_absolute_error: 0.0382
Epoch 228/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0390 - mean_absolute_error: 0.0390
Epoch 229/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0384 - mean_absolute_error: 0.0384
Epoch 230/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0391 - mean_absolute_error: 0.0391
Epoch 231/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0381 - mean_absolute_error: 0.0381
Epoch 232/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0388 - mean_absolute_error: 0.0388
Epoch 233/500
1/1 [===

1/1 [==============================] - 0s 5ms/step - loss: 0.0375 - mean_absolute_error: 0.0375
Epoch 299/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0377 - mean_absolute_error: 0.0377
Epoch 300/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0375 - mean_absolute_error: 0.0375
Epoch 301/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0371 - mean_absolute_error: 0.0371
Epoch 302/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0376 - mean_absolute_error: 0.0376
Epoch 303/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0367 - mean_absolute_error: 0.0367
Epoch 304/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0377 - mean_absolute_error: 0.0377
Epoch 305/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0373 - mean_absolute_error: 0.0373
Epoch 306/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0372 - mean_absolute_error: 0.0372
Epoch 307/500
1/1 [=====

1/1 [==============================] - 0s 6ms/step - loss: 0.0355 - mean_absolute_error: 0.0355
Epoch 373/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0354 - mean_absolute_error: 0.0354
Epoch 374/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0350 - mean_absolute_error: 0.0350
Epoch 375/500
1/1 [==============================] - 0s 11ms/step - loss: 0.0354 - mean_absolute_error: 0.0354
Epoch 376/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0350 - mean_absolute_error: 0.0350
Epoch 377/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0357 - mean_absolute_error: 0.0357
Epoch 378/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0351 - mean_absolute_error: 0.0351
Epoch 379/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0357 - mean_absolute_error: 0.0357
Epoch 380/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0352 - mean_absolute_error: 0.0352
Epoch 381/500
1/1 [====

1/1 [==============================] - 0s 5ms/step - loss: 0.0348 - mean_absolute_error: 0.0348
Epoch 447/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0345 - mean_absolute_error: 0.0345
Epoch 448/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0347 - mean_absolute_error: 0.0347
Epoch 449/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0347 - mean_absolute_error: 0.0347
Epoch 450/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0337 - mean_absolute_error: 0.0337
Epoch 451/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0342 - mean_absolute_error: 0.0342
Epoch 452/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0340 - mean_absolute_error: 0.0340
Epoch 453/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0346 - mean_absolute_error: 0.0346
Epoch 454/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0338 - mean_absolute_error: 0.0338
Epoch 455/500
1/1 [====

In [32]:
preds=model.predict(ret)

In [33]:
weights_final[2]

array([0.2722318 , 0.00419365, 0.39494786, 0.03044642, 0.29818028])

In [34]:
#Train with more data and higher look back for possibly better results

preds[2]

array([0.287212  , 0.00283758, 0.26780105, 0.04747205, 0.23260981],
      dtype=float32)